In [344]:
import pandas as pd
import re

In [345]:
stop_times = pd.read_csv("/home/invicta117/Downloads/gtfs/stop_times.txt", delimiter=",")
stops = pd.read_csv("/home/invicta117/Downloads/gtfs/stops.txt", delimiter=",")
routes = pd.read_csv("/home/invicta117/Downloads/gtfs/routes.txt", delimiter=",")
trips = pd.read_csv("/home/invicta117/Downloads/gtfs/trips.txt", delimiter=",")
calendar = pd.read_csv("/home/invicta117/Downloads/gtfs/calendar.txt", delimiter=",")

In [346]:
stop_times["arrival_time"] = pd.to_timedelta(stop_times["arrival_time"])

In [347]:
departure = "16:02"
origin = "Clonskeagh Road, stop 878"
destination = "Dawson Street, stop 793"
day = "thursday"
bus_route = "11"

In [348]:
days = list(calendar[calendar[day] == 1]["service_id"].values)

In [349]:
day_trips = set(trips[trips["service_id"].isin(days)]["trip_id"].values)

In [350]:
route_trips = set(trips[trips["route_id"].isin(list(routes[routes["route_short_name"] == bus_route]["route_id"].values))]["trip_id"].values)

In [351]:
start_trips = stops[stops["stop_name"] == origin]["stop_id"].values[0]
end_trips = stops[stops["stop_name"] == destination]["stop_id"].values[0]

In [352]:
start_trips  = set(stop_times[stop_times["stop_id"] == start]["trip_id"].values)
end_trips  = set(stop_times[stop_times["stop_id"] == end]["trip_id"].values)

In [353]:
r = start_trips.intersection(end_trips)
r = r.intersection(route_trips)
r = r.intersection(day_trips)

In [354]:
I = None
O = None

for trip in r:
    if re.search(".I$", trip):
        I = trip
        break
        
for trip in r:
    if re.search(".O$", trip):
        O = trip
        break
print(O)
print(I)

None
2026.y1005.60-11-d12-1.16.I


In [355]:
if O != None and ((stop_times[(stop_times["trip_id"] == O) & (stop_times["stop_id"] == start)]["stop_sequence"].values[0]) < (stop_times[(stop_times["trip_id"] == O) & (stop_times["stop_id"] == end)]["stop_sequence"].values[0])):
    correct_direction = [trip for trip in r if re.search(".O$", trip)]
elif I != None and ((stop_times[(stop_times["trip_id"] == I) & (stop_times["stop_id"] == start)]["stop_sequence"].values[0]) < (stop_times[(stop_times["trip_id"] == I) & (stop_times["stop_id"] == end)]["stop_sequence"].values[0])):
    correct_direction = [trip for trip in r if re.search(".I$", trip)]

In [356]:
final = stop_times[(stop_times["trip_id"].isin(correct_direction)) & (stop_times["stop_id"] == start)][["trip_id","arrival_time"]]
final["arrival_time"] = abs(final["arrival_time"] - pd.to_timedelta(departure+':00'))
final = final.sort_values(by="arrival_time")

In [357]:
final

,trip_id,arrival_time
17898,2091.y1005.60-11-d12-1.16.I,0 days 00:00:07
709429,1991.y1003.60-11-b12-1.18.I,0 days 00:00:07
709366,1985.y1003.60-11-b12-1.18.I,0 days 00:14:53
17835,2085.y1005.60-11-d12-1.16.I,0 days 00:14:53
17772,2058.y1005.60-11-d12-1.16.I,0 days 00:15:07
...,...,...
17583,2018.y1005.60-11-d12-1.16.I,0 days 08:39:57
714784,2001.y1003.60-11-b12-1.18.I,0 days 08:59:57
17709,2101.y1005.60-11-d12-1.16.I,0 days 08:59:57
714721,1930.y1003.60-11-b12-1.18.I,0 days 09:19:57


In [358]:
tripid = final.iloc[0]["trip_id"]

In [359]:
seq = stop_times[stop_times["trip_id"] == tripid][["stop_id", "stop_sequence"]].sort_values(by="stop_sequence")

In [360]:
rstart = seq[seq["stop_id"] == start].index[0]

In [361]:
rfin = seq[seq["stop_id"] == end].index[0]

In [362]:
test = seq.loc[rstart:rfin]

In [363]:
final = test.merge(stops, how='left', on='stop_id')

In [364]:
flist = list(final["stop_name"].values)
flist = [stop.split(" stop ")[1] for stop in flist]
flist

['878',
 '879',
 '880',
 '881',
 '882',
 '883',
 '884',
 '885',
 '886',
 '887',
 '903',
 '904',
 '906',
 '907',
 '908',
 '909',
 '786',
 '793']